In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import cm

In [ ]:
cwd = os.getcwd()
root_dir = os.path.abspath(os.path.join(cwd, os.pardir))
root_dir

In [ ]:
def read_sump(sump_name="sump.out"):
    full_name = os.path.join(root_dir, sump_name)
    if not os.path.isfile(full_name):
        print("El archivo %s no existe"%os.path.basename(full_name))
        return
    df = pd.read_csv(full_name, delimiter='\s+', header=None,
                      names=["idx",
                             "bad",
                             "t_t",
                             "L_t", # Total
                             "mass", # Particle
                             "a_i", "e_i", "M_i", "omega_i", "R_i",
                             "l_i",
                             "t_f",
                             "a_f", "e_f", "M_f", "omega_f", "R_f",
                             "l_f",
                             "amin", "amax", 
                             "emin", "emax", 
                             "da", "de"])
    if df.idx.nunique() == 1:
        df.idx = range(0,len(df))
    return df

def read_outfile(name, nboulders=2, coord=False):
    full_name = os.path.join(root_dir, name)
    if not os.path.isfile(full_name):
        print("El archivo %s no existe"%os.path.basename(full_name))
        return
    if coord:
        df = pd.read_csv(name, delimiter='\s+', header=None,
                         names=["idx",
                                "t",
                                "x", "y",
                                "vx", "vy",
                                "ax", "ay",
                                "mass", "radio"])
        df.name = name.split(".")[0]
        df.nt = df.idx.nunique()
        df.nb = nboulders
        df.np = df.nt - nboulders
        df["r"] = np.sqrt(df["x"]**2 + df["y"]**2)
        df["v"] = np.sqrt(df["vx"]**2 + df["vy"]**2)
        df["a"] = np.sqrt(df["ax"]**2 + df["ay"]**2)
    else:
        df = pd.read_csv(name, delimiter='\s+', header=None,
                         names=["idx",
                                "t",
                                "a", "e", "M", "omega", "MMR",
                                "mass", # Particle
                                "acorot", # To Be Changed with Particle_distance
                                "spin", "mast"]) # Asteroid
        df.name = name.split(".")[0]
        df["lorb"] = np.sqrt(df.a * (1 - df.e**2))
        df.da = df.a.max() - df.a.min()
        df.de = df.e.max() - df.e.min()
        df.tmax = df.t.iloc[-1]
    return df

def read_map(name):
    full_name = os.path.join(root_dir, name)
    if not os.path.isfile(full_name):
        print("El archivo %s no existe"%os.path.basename(full_name))
        return
    df = pd.read_csv(name, delimiter='\s+', header=None,
                     names=["x", "y", "pot", "ax", "ay"])
    df["r"] = np.sqrt(df["x"]**2 + df["y"]**2)
    df["acc"] = np.sqrt(df["ax"]**2 + df["ay"]**2)
    return df

get_MMR = lambda a: (a/2.130962668815482E+02)**(3/2)
get_a = lambda R: R**(2/3.) * 2.130962668815482E+02

In [ ]:
sump = read_sump()
sump

In [ ]:
if sump:
    bad = sump.bad > 0
    plt.figure(dpi=150)
    plt.suptitle("$t_{max} = %d$ yrs"%int(sump.t_t.max()/365.25))
    plt.plot(sump.R_i[bad], sump.R_f[bad], ".", label="Colision")
    plt.plot(sump.R_i[~bad], sump.R_f[~bad], ".", label="Survived")
    plt.xlabel("Initial RMM")
    plt.ylabel("Final RMM")
    plt.legend()
    plt.show()

In [ ]:
file_list = []
pref = "dpy"
suffix = ""
# Recorre todas las subcarpetas en la carpeta raíz
for subdir in os.listdir(root_dir):
    # Verifica si el nombre de la subcarpeta comienza con 'pref'
    if subdir.startswith(pref):
        subdir_path = os.path.join(root_dir, subdir)
        # Recorre todos los archivos en la subcarpeta
        for filename in os.listdir(subdir_path):
            # Verifica si el nombre del archivo comienza con
            # "chaos" y termina con ".dat"
            if (filename.startswith("salida")
                and filename.endswith(".dat")
                and (suffix in filename)
            ):
                filepath = os.path.join(subdir_path, filename)
                file_list.append(filepath)

# Ordena los nombres de los archivos por el valor de i en "chaos%d%s.dat"
if suffix == "":
    file_list = sorted(
        file_list, key=lambda x: int(x.split("salida")[1].split(".dat")[0])
    )
else:
    file_list = sorted(
        file_list,
        key=lambda x: int(
            x.split("salida")[1].split(".dat")[0].split(suffix)[0]
        ),
    )

In [ ]:
dfs = dict()
for salida in file_list:
    number = os.path.basename(salida)[6:].split(".")[0]
    df = read_outfile(salida, coord=False)
    dfs[int(number)] = df    

In [ ]:
if dfs:
    plt.figure(dpi=200)
    plt.suptitle("Drag coefficient: %.3f"%1e-2)
    for idf in dfs:
        plt.plot(dfs[idf].t/365.25, dfs[idf].e, "k-", lw=0.6, ms=0.2, alpha=idf/120)
    plt.xlabel("Years")
    plt.ylabel("$e$")
    plt.semilogx()
    plt.semilogy()
    plt.show()

In [ ]:
if dfs:
    plt.figure(dpi=170)
    plt.suptitle("Drag coefficient: %.3f"%1e-2)
    for idf in dfs:
        plt.plot(dfs[idf].t/365.25, (dfs[idf].lorb - dfs[idf].lorb.iloc[0])/dfs[idf].lorb.iloc[0], "k.", lw=0.6, ms=0.2)
    plt.xlabel("Years")
    plt.ylabel("$\\Delta L_{part}/L_{part}(t=0)$")
    plt.grid()
    plt.semilogx()
    plt.xticks([10**x for x in range(-2,7)])
    plt.show()

In [ ]:
if dfs:
    G = 4.9823394e-10 
    m0 = 6.3e18
    R0 = 129.
    mD = m0 * 1e1
    mp = mD / len(dfs)
    mu = G * m0 * mp
    I = 4.204006931643356E+22
    Last = 7.575367962011342E+23
    omegai = 1.801939931400187E+01

In [ ]:
if dfs:
    plt.figure(dpi=150)
    for i in range(len(dfs)-2):
        Lparti = np.sqrt(mu) * dfs[i].lorb.iloc[0]
        dfs[i].Ltot = Last + Lparti
        plt.plot(dfs[i].a.iloc[0], Lparti / dfs[i].Ltot, "k.")
        Lpartf = np.sqrt(mu) * dfs[i].lorb.iloc[-1]
        plt.plot(dfs[i].a.iloc[0], Lpartf / dfs[i].Ltot, "r.")
    i = len(dfs)-1
    Lparti = np.sqrt(mu) * dfs[i].lorb.iloc[0]
    dfs[i].Ltot = Last + Lparti
    plt.plot(dfs[i].a.iloc[0], Lparti / dfs[i].Ltot, "k.", label="Initial")
    Lpartf = np.sqrt(mu) * dfs[i].lorb.iloc[-1]
    plt.plot(dfs[i].a.iloc[0], Lpartf / dfs[i].Ltot, "r.", label="Final")
    plt.legend()
    plt.ylabel("$L_{part}/L_{tot}$")
    plt.xlabel("Initial $a$ [km]")
    plt.show()

In [ ]:
if dfs:
    plt.figure(dpi=170)
    for i in dfs:
        df = dfs[i]
        Li = df.lorb.iloc[0]
        Lf = df.lorb.iloc[-1]
        plt.plot(df.tmax/365.25, np.sqrt(mu) * (Lf - Li) / I, "k.")
    plt.xlabel("tmax [years]")
    plt.ylabel("$\Delta a$ [km]")
    plt.ylabel("$\Delta \Omega$")
    plt.semilogx()
    # plt.semilogy()
    plt.show()

In [ ]:
if dfs:
    plt.figure(dpi=170)
    for i in dfs:
        df = dfs[i]
        Li = df.lorb.iloc[0]
        Lf = df.lorb.iloc[-1]
        delta_omega = np.sqrt(mu) * (Lf - Li) / I
        omegaf = omegai + delta_omega
        plt.plot(df.tmax/365.25, 2 * np.pi * (1/omegaf - 1/omegai), "k.")
    plt.xlabel("tmax [years]")
    plt.ylabel("$\Delta a$ [km]")
    plt.ylabel("$\Delta$ P [days]")
    plt.semilogx()
    # plt.semilogy()
    plt.show()

In [ ]:
if dfs:
    plt.figure(dpi=170)
    for i in dfs:
        df = dfs[i]
        Li = df.lorb.iloc[0]
        Lf = df.lorb.iloc[-1]
        plt.plot(df.tmax/365.25, np.sqrt(mu) * (Lf - Li), "k.")
    plt.xlabel("tmax [years]")
    plt.ylabel("$\Delta a$ [km]")
    plt.ylabel("$\Delta L$")
    plt.semilogx()
    plt.semilogy()
    plt.show()

In [ ]:
mapa = read_map("mapa.dat")
mapa

In [ ]:
if mapa:
    # Create pivot table
    pivot = mapa.pivot_table(values=["pot", "acc", "ax", "ay"], index="x", columns="y")
    pivot

In [ ]:
if mapa:
    # Plot the potential, with pcolormesh
    plt.figure(dpi=150)
    plt.pcolormesh(np.log10(pivot["pot"].abs()))
    plt.colorbar()
    plt.show()

In [ ]:
if mapa:
    # Plot the potential, with pcolormesh
    fig, axs = plt.subplots(1,3,dpi=150, sharey=True, figsize=(16,4))
    axs[0].pcolormesh(np.log10(pivot["acc"].abs()), vmin=1, vmax=7)
    axs[1].pcolormesh(np.log10(pivot["ax"].abs()), vmin=1, vmax=7)
    p = axs[2].pcolormesh(np.log10(pivot["ay"].abs()), vmin=1, vmax=7)
    fig.colorbar(p, ax=axs.ravel().tolist())
    plt.show()